# Kaggle Quora Challenge

by Seth Rabin and Mukul Ramm

The goal is to determine whether two questions have the same purpose

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
data_dir = '../../data/'

In [9]:
train = pd.read_csv(data_dir + 'train.csv', index_col = 'id')
train.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [12]:
test = pd.read_csv(data_dir + 'test.csv', index_col = 'test_id')
test.head()

C:\Users\Seth Rabin\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,question1,question2
test_id,,
0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,What but is the best way to send money from Ch...,What you send money to China?
3,Which food not emulsifiers?,What foods fibre?
4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [16]:
train.iloc[0]

qid1                                                            1
qid2                                                            2
question1       What is the step by step guide to invest in sh...
question2       What is the step by step guide to invest in sh...
is_duplicate                                                    0
Name: 0, dtype: object

In [17]:
def multiply

SyntaxError: invalid syntax (<ipython-input-17-09220f504c2e>, line 1)

In [19]:
def multiply (x):
    return x*2
train.is_duplicate.apply(multiply)

id
0         0
1         0
2         0
3         0
4         0
5         2
6         0
7         2
8         0
9         0
10        0
11        2
12        2
13        2
14        0
15        2
16        2
17        0
18        2
19        0
20        2
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        2
         ..
404260    0
404261    2
404262    0
404263    0
404264    0
404265    2
404266    0
404267    2
404268    0
404269    0
404270    0
404271    0
404272    2
404273    2
404274    2
404275    0
404276    0
404277    0
404278    0
404279    0
404280    2
404281    2
404282    2
404283    0
404284    2
404285    0
404286    2
404287    0
404288    0
404289    0
Name: is_duplicate, Length: 404290, dtype: int64

In [21]:
train.iloc[0].question1

'What is the step by step guide to invest in share market in india?'

In [65]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404290 entries, 0 to 404289
Data columns (total 6 columns):
qid1            404290 non-null int64
qid2            404290 non-null int64
question1       404290 non-null object
question2       404288 non-null object
is_duplicate    404290 non-null int64
unique          404290 non-null int64
dtypes: int64(4), object(2)
memory usage: 41.6+ MB


# Feature Engineer

The features I plan to engineer are -

1. Frequency of words
2. Synonyms
3. Similar words
4. Unique words
5. Speech tagging



In [62]:
import nltk
from nltk.corpus import stopwords
import string


In [30]:
dummy = train.iloc[0].question1

In [51]:
dummy1 = train.iloc[0].question1

In [52]:
dummy2 = train.iloc[0].question2

In [49]:
def process(x):
    import nltk
    import string 
    from nltk import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem.wordnet import WordNetLemmatizer
    
    stop_words = stopwords.words('english') + list(string.punctuation)
    
    tokens = word_tokenize(x)
    updated = list(set(tokens) - set(stop_words))
    
    lemma = WordNetLemmatizer()
    
    lemmatized = [lemma.lemmatize(word) for word in updated]
    
    return lemmatized

In [56]:
train.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [58]:
def create_unique(row):
    proc1 = process(row.question1)
    proc2 = process(row.question2)
    
    return 1 if (set(proc1) - set(proc2) or set(proc2) - set(proc1)) else 0

In [66]:
train['unique'] = 0
train.unique = train.apply(create_unique, axis=1)
train.head()

TypeError: ('expected string or bytes-like object', 'occurred at index 105780')

In [ ]:
test['unique'] = 0
test.unique = test.apply(create_unique, axis=1)
test.head()